<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/TFPractice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype("float32"), x_test.astype("float32")
x_train,x_test = x_train.reshape([-1,784]), x_test.reshape([-1,784])
x_train, x_test = x_train/255., x_test/255.

11501568/11490434 [==============================] - 0s 0us/step


In [2]:
learning_rate = 0.02
training_epochs = 50
batch_size = 256
display_step = 1
examples_to_show = 10
input_size = 784
hidden1_size = 256
hidden2_size = 128

#비지도학습이기 때문에 y데이터는 묶어주지 않음.
train_data = tf.data.Dataset.from_tensor_slices(x_train)
train_data = train_data.shuffle(60000).batch(batch_size)

def random_normal_initializer_with_stddev_v1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

class AutoEncoder(tf.keras.Model):
  def __init(self):
